In [ ]:
pip install pandas numpy scikit-learn matplotlib


In [ ]:
pip install xgboost

In [ ]:
#tuning the model with 5 configs of XGBoost algorithm with different depth & learning rate combos.
import pandas as pd, numpy as np
from pathlib import Path
import sys
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor

# -------------------------
# CONFIG
# -------------------------
RS_FILENAME  = "RS_monthly.csv"         # Remote sensing table (from GEE)
Q_FILENAME   = "discharge_monthly.csv"  # Monthly discharge table
OUT_DIR      = "outputs"
CUTOFF_DATE  = "2018-01-01"             # Train < cutoff, Test >= cutoff
MIN_TEST_ROWS_PER_STATION = 6

# RMSE helper (future-proof for old/new sklearn versions)
try:
    from sklearn.metrics import root_mean_squared_error as rmse_func
except ImportError:
    def rmse_func(y_true, y_pred):
        return mean_squared_error(y_true, y_pred, squared=False)

# Hydrology metric: Nash–Sutcliffe Efficiency
def nse(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return 1 - ((y_true - y_pred)**2).sum() / ((y_true - y_true.mean())**2).sum()

# -------------------------
# Utility functions
# -------------------------
def std_name(s):
    """Normalize station names (fix whitespace & known typos)."""
    return " ".join(str(s).split()).replace("Soucherres", "Soucheres")

def find_col(candidates, cols):
    """Find a column name in `cols` matching one of `candidates` (exact or prefix/suffix)."""
    cols = list(cols)
    for c in candidates:
        if c in cols: return c
    for col in cols:
        for c in candidates:
            if col.lower().startswith(c.lower()) or col.lower().endswith(c.lower()):
                return col
    return None

# -------------------------
# Load Remote Sensing data
# -------------------------
rs = pd.read_csv(RS_FILENAME)

# Normalize date column (sometimes called `date_str`)
if "date" not in rs.columns and "date_str" in rs.columns:
    rs = rs.rename(columns={"date_str": "date"})
rs["date"] = pd.to_datetime(rs["date"], errors="coerce")

# Find precipitation and NDVI columns (names may vary from GEE export)
p_col = find_col(["P_mm","precip","rain_mm","precip_mm"], rs.columns)
n_col = find_col(["NDVI"], rs.columns)
rs = rs[["station_id","date",p_col,n_col]].rename(columns={p_col:"P_mm", n_col:"NDVI"})

# -------------------------
# Load Discharge (Q) data
# -------------------------
q = pd.read_csv(Q_FILENAME, parse_dates=["date"])[["station_id","date","Q_m3s"]]
q["station_id"]  = q["station_id"].map(std_name)
rs["station_id"] = rs["station_id"].map(std_name)

# -------------------------
# Merge datasets (keep only rows where both Q and RS exist)
# -------------------------
df = (q.merge(rs, on=["station_id","date"], how="inner")
        .sort_values(["station_id","date"])
        .dropna(subset=["Q_m3s","P_mm","NDVI"])
        .reset_index(drop=True))

# -------------------------
# Add lags & rolling features (catchment memory)
# -------------------------
def add_lags_rollups(g):
    # Precipitation memory
    g["P_mm_lag1"] = g["P_mm"].shift(1)
    g["P_mm_lag2"] = g["P_mm"].shift(2)
    g["P_mm_lag3"] = g["P_mm"].shift(3)
    g["P_3mo_sum"] = g["P_mm"].rolling(3).sum()
    g["P_6mo_sum"] = g["P_mm"].rolling(6).sum()
    # Vegetation / wetness memory
    g["NDVI_lag1"]   = g["NDVI"].shift(1)
    g["NDVI_lag2"]   = g["NDVI"].shift(2)
    g["NDVI_3mo_mn"] = g["NDVI"].rolling(3).mean()
    return g

df = df.groupby("station_id", group_keys=False).apply(add_lags_rollups)
df = df.dropna().reset_index(drop=True)

# -------------------------
# Add seasonality (smooth month encoding)
# -------------------------
df["month"] = df["date"].dt.month
df["month_sin"] = np.sin(2*np.pi*(df["month"]/12))
df["month_cos"] = np.cos(2*np.pi*(df["month"]/12))

# -------------------------
# Train/Test split (time-aware)
# -------------------------
cutoff = pd.Timestamp(CUTOFF_DATE)
train, test = df[df["date"] < cutoff], df[df["date"] >= cutoff]

features = [
    "P_mm","NDVI","month_sin","month_cos",
    "P_mm_lag1","P_mm_lag2","P_mm_lag3",
    "P_3mo_sum","P_6mo_sum",
    "NDVI_lag1","NDVI_lag2","NDVI_3mo_mn"
]

X_tr, y_tr = train[features], train["Q_m3s"]
X_te, y_te = test[features],  test["Q_m3s"]

# -------------------------
# Parameter tuning sweep
# -------------------------
param_grid = [
    # Lower depth = simpler trees (less overfit), higher = more complex
    {"max_depth":4, "learning_rate":0.05, "subsample":0.8, "colsample_bytree":0.8},
    {"max_depth":6, "learning_rate":0.05, "subsample":0.8, "colsample_bytree":0.8},
    {"max_depth":8, "learning_rate":0.05, "subsample":0.8, "colsample_bytree":0.8},
    # Try smaller learning rate with more stochastic sampling
    {"max_depth":6, "learning_rate":0.03, "subsample":0.9, "colsample_bytree":0.9},
    # Try faster learning rate (may overfit but worth testing)
    {"max_depth":6, "learning_rate":0.1,  "subsample":0.8, "colsample_bytree":0.8},
]

results = []
for i, params in enumerate(param_grid, 1):
    model = XGBRegressor(
        n_estimators=800,      # number of trees (enough for convergence)
        reg_lambda=1.0,        # L2 regularization
        random_state=42,
        n_jobs=-1,
        tree_method="hist",    # fast CPU mode
        **params               # inject params from the grid
    )
    # Train and evaluate
    model.fit(X_tr, y_tr)
    pred = model.predict(X_te)
    results.append({
        "config": params,
        "R2": r2_score(y_te, pred),
        "RMSE": rmse_func(y_te, pred),
        "NSE": nse(y_te, pred)
    })
    print(f"Config {i}: {params} → NSE={results[-1]['NSE']:.3f}, R2={results[-1]['R2']:.3f}")

# -------------------------
# Save leaderboard
# -------------------------
resdf = pd.DataFrame(results).sort_values("NSE", ascending=False)
Path(OUT_DIR).mkdir(exist_ok=True)
resdf.to_csv(Path(OUT_DIR)/"xgb_tuning_results.csv", index=False)

print("\nBest config:\n", resdf.head(1))


In [ ]:
from IPython.display import Image, display

display(Image(filename="outputs/obs_vs_pred.png"))
display(Image(filename="outputs/feature_importance.png"))


In [ ]:
#training the model wit the best parameter from 5 configs of XGBoost with different depth & learning rate combos.
# Finalize best XGB config and save outputs
import pandas as pd, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor

# ---------- paths ----------
RS_FILENAME  = "RS_monthly.csv"
Q_FILENAME   = "discharge_monthly.csv"
OUT_DIR      = Path("outputs"); OUT_DIR.mkdir(exist_ok=True)
CUTOFF_DATE  = "2018-01-01"

# ---------- helpers ----------
try:
    from sklearn.metrics import root_mean_squared_error as rmse_func
except ImportError:
    def rmse_func(y_true, y_pred):
        return mean_squared_error(y_true, y_pred, squared=False)

def nse(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return 1 - ((y_true - y_pred)**2).sum() / ((y_true - y_true.mean())**2).sum()

def std_name(s): return " ".join(str(s).split()).replace("Soucherres", "Soucheres")
def find_col(cands, cols):
    cols = list(cols)
    for c in cands:
        if c in cols: return c
    for col in cols:
        for c in cands:
            if col.lower().startswith(c.lower()) or col.lower().endswith(c.lower()):
                return col
    return None

# ---------- load RS ----------
rs = pd.read_csv(RS_FILENAME)
if "date" not in rs.columns and "date_str" in rs.columns:
    rs = rs.rename(columns={"date_str": "date"})
rs["date"] = pd.to_datetime(rs["date"], errors="coerce")
p_col = find_col(["P_mm","precip","rain_mm","precip_mm"], rs.columns)
n_col = find_col(["NDVI"], rs.columns)
rs = rs[["station_id","date",p_col,n_col]].rename(columns={p_col:"P_mm", n_col:"NDVI"})

# ---------- load Q ----------
q = pd.read_csv(Q_FILENAME, parse_dates=["date"])[["station_id","date","Q_m3s"]]
q["station_id"]  = q["station_id"].map(std_name)
rs["station_id"] = rs["station_id"].map(std_name)

# ---------- merge ----------
df = (q.merge(rs, on=["station_id","date"], how="inner")
        .sort_values(["station_id","date"])
        .dropna(subset=["Q_m3s","P_mm","NDVI"])
        .reset_index(drop=True))

# ---------- lags/rollups ----------
def add_lags_rollups(g):
    g["P_mm_lag1"] = g["P_mm"].shift(1)
    g["P_mm_lag2"] = g["P_mm"].shift(2)
    g["P_mm_lag3"] = g["P_mm"].shift(3)
    g["P_3mo_sum"] = g["P_mm"].rolling(3).sum()
    g["P_6mo_sum"] = g["P_mm"].rolling(6).sum()
    g["NDVI_lag1"]   = g["NDVI"].shift(1)
    g["NDVI_lag2"]   = g["NDVI"].shift(2)
    g["NDVI_3mo_mn"] = g["NDVI"].rolling(3).mean()
    return g

# if your pandas >=2.1 you can add include_groups=False
df = df.groupby("station_id", group_keys=False).apply(add_lags_rollups)
df = df.dropna().reset_index(drop=True)

# ---------- seasonality ----------
df["month"] = df["date"].dt.month
df["month_sin"] = np.sin(2*np.pi*(df["month"]/12))
df["month_cos"] = np.cos(2*np.pi*(df["month"]/12))

# ---------- split ----------
cutoff = pd.Timestamp(CUTOFF_DATE)
train, test = df[df["date"] < cutoff], df[df["date"] >= cutoff]

features = [
    "P_mm","NDVI","month_sin","month_cos",
    "P_mm_lag1","P_mm_lag2","P_mm_lag3",
    "P_3mo_sum","P_6mo_sum",
    "NDVI_lag1","NDVI_lag2","NDVI_3mo_mn"
]
X_tr, y_tr = train[features], train["Q_m3s"]
X_te, y_te = test[features],  test["Q_m3s"]

# ---------- best params from your sweep ----------
best = dict(max_depth=6, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8)

model = XGBRegressor(
    n_estimators=1500,  # a bit larger than sweep for smoother fit
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    **best
)
model.fit(X_tr, y_tr)

pred = model.predict(X_te)
rmse = rmse_func(y_te, pred)
r2   = r2_score(y_te, pred)
nse_ = nse(y_te, pred)

with open(OUT_DIR/"model_report.txt","w") as f:
    f.write(f"XGB tuned — test R2 = {r2:.3f}\n")
    f.write(f"XGB tuned — test RMSE = {rmse:.3f}\n")
    f.write(f"XGB tuned — test NSE = {nse_:.3f}\n")

print(f"XGB tuned — R2={r2:.3f}  RMSE={rmse:.3f}  NSE={nse_:.3f}")

# ---------- example plot ----------
sid = test["station_id"].iloc[0]
ex = test[test["station_id"]==sid].copy().sort_values("date")
ex["pred"] = model.predict(ex[features])

plt.figure(figsize=(9,4))
plt.plot(ex["date"], ex["Q_m3s"], label="Observed")
plt.plot(ex["date"], ex["pred"], label="Predicted")
plt.title(f"Discharge — {sid} (test)")
plt.xlabel("Date"); plt.ylabel("Q (m³/s)"); plt.legend(); plt.tight_layout()
plt.savefig(OUT_DIR/"obs_vs_pred.png", dpi=150); plt.close()

# ---------- per-station skill ----------
rows = []
for sid, g in test.groupby("station_id"):
    if len(g) < 6: 
        continue
    p = model.predict(g[features])
    rows.append({
        "station_id": sid,
        "R2": r2_score(g["Q_m3s"], p),
        "RMSE": rmse_func(g["Q_m3s"], p),
        "NSE": nse(g["Q_m3s"], p)
    })
pd.DataFrame(rows).sort_values("NSE", ascending=False).to_csv(OUT_DIR/"station_skill.csv", index=False)

# ---------- feature importance ----------
imp = pd.Series(model.feature_importances_, index=features).sort_values(ascending=True)

plt.figure(figsize=(7,5))
imp.plot(kind="barh")
plt.title("Feature importance (XGBoost — tuned)")
plt.tight_layout()
plt.savefig(OUT_DIR/"feature_importance.png", dpi=150); plt.close()
